Southern University of Science and Technology-Department of Computer Science and Engineering

Course: Machine Learning(CS 405)-Professor: Qi Hao

## Homework #2
#### Due date: October, 7th, 2019

In [1]:
"""
Import libraries that you might require.

"""
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
import math
import matplotlib.pyplot as plt
import operator
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

We will implement the KNN algorithm for the breast cancer dataset. Refer to the pdf and the following functions for the instructions. Complete all the functions as indicated below. The four functions would be autograded as mentioned in the pdf.

In [2]:
"""
Task 1: Classification

Please implement KNN for K: 3, 5, and 7 with the following norms:
L1
L2
L-inf
"""

# Read data (Breast Cancer Dataset). Remember to comment out the code not contained in a function.
from sklearn.datasets import load_breast_cancer

breast = load_breast_cancer()

X = breast['data']
y = breast['target']

np.random.seed(100)
p = np.random.permutation(len(X))
X, y = X[p], y[p]

X_train, y_train = X[:400], y[:400]
X_val, y_val = X[400:500], y[400:500]
X_test, y_test = X[500:], y[500:]
min_max_scaler = preprocessing.MinMaxScaler()
X_train, X_val, X_test = min_max_scaler.fit_transform(X_train), min_max_scaler.fit_transform(X_val), min_max_scaler.fit_transform(X_test)

def distanceFunc(metric_type, vec1, vec2):
    """
    Computes the distance between two d-dimension vectors. 
    
    Please DO NOT use Numpy's norm function when implementing this function. 
    
    Args:
        metric_type (str): Metric: L1, L2, or L-inf
        vec1 ((d,) np.ndarray): d-dim vector
        vec2 ((d,)) np.ndarray): d-dim vector
    
    Returns:
        distance (float): distance between the two vectors
    """

    diff = vec1 - vec2
    diff = np.array([abs(x) for x in diff])
    #print(len(diff))
    if metric_type == "L1":
        distance = sum(diff)

    if metric_type == "L2":
        distance = np.sqrt(sum(diff * diff)) 
        
    if metric_type == "L-inf":
        distance = max([x for x in diff])
        
    return distance


def computeDistancesNeighbors(K, metric_type, X_train, y_train, sample):
    """
    Compute the distances between every datapoint in the train_data and the 
    given sample. Then, find the k-nearest neighbors.
    
    Return a numpy array of the label of the k-nearest neighbors.
    
    Args:
        K (int): K-value
        metric_type (str): metric type
        X_train ((n,p) np.ndarray): Training data with n samples and p features
        y_train : Training labels
        sample ((p,) np.ndarray): Single sample whose distance is to computed with every entry in the dataset
        
    Returns:
        neighbors (list): K-nearest neighbors' labels
    """
    
    # You will also call the function "distanceFunc" here
    # Complete this function
    neighbors = []
    neighbors_distance = []
    for i in range(len(X_train)):
        curr_distance = distanceFunc(metric_type, X_train[i], sample)
        neighbors_distance.append(curr_distance)
    neighbors_distance = np.array(neighbors_distance)
    sortedDistIndicies = neighbors_distance.argsort()
    count = 0
    while count < K:
        neighbors.append(y_train[sortedDistIndicies[count]])
        count += 1
    return neighbors


def Majority(neighbors):
    """
    Performs majority voting and returns the predicted value for the test sample.
    
    Since we're performing binary classification the possible values are [0,1].
    
    Args:
        neighbors (list): K-nearest neighbors' labels
        
    Returns:
        predicted_value (int): predicted label for the given sample
    """
    
    # Performs majority voting
    # Complete this function
    value_0, value_1 = 0, 0
    for label in neighbors:
        if label:
            value_1 += 1
        else:
            value_0 += 1
    predicted_value = 0 if value_0 > value_1 else 1
    return predicted_value


def KNN(K, metric_type, X_train, y_train, X_val):
    """
    Returns the predicted values for the entire validation or test set.
    
    Please DO NOT use Scikit's KNN model when implementing this function. 

    Args:
        K (int): K-value
        metric_type (str): metric type
        X_train ((n,p) np.ndarray): Training data with n samples and p features
        y_train : Training labels
        X_val ((n, p) np.ndarray): Validation or test data
        
    Returns:
        predicted_values (list): output for every entry in validation/test dataset 
    """
    
    # Complete this function
    # Loop through the val_data or the test_data (as required)
    # and compute the output for every entry in that dataset  
    # You will also call the function "Majority" here
    predictions = []
    for i in range(len(X_val)):
        neighbors_label = computeDistancesNeighbors(K, metric_type, X_train, y_train, X_val[i])
        predictions.append(Majority(neighbors_label))
    return predictions


def evaluation(predicted_values, actual_values):
    """
    Computes the accuracy of the given datapoints.
    
    Args:
        predicted_values ((n,) np.ndarray): Predicted values for n samples
        actual_values ((n,) np.ndarray): Actual values for n samples
    
    Returns:
        accuracy (float): accuracy
    """
    count = 0
    for i in range(len(predicted_values)):
        if predicted_values[i] == actual_values[i]:
            count += 1
        
    return count / len(predicted_values)


def main():
    """
    Calls the above functions in order to implement the KNN algorithm.
    
    Test over the following range K = 3,5,7 and all three metrics. 
    In total you will have nine combinations to try.
    
    PRINTS out the accuracies for the nine combinations on the validation set,
    and the accuracy on the test set for the selected K value and appropriate norm.
    
    REMEMBER: You have to report these values by populating the Table 1.
    """
    
    ## Complete this function
    
    K = [3,5,7]
    norm = ["L1", "L2", "L-inf"]
    acc_validation = []
    print("<<<<VALIDATION DATA PREDICTIONS>>>>")
    for k in K:
        for nor in norm:
            acc = evaluation(KNN(k,nor,X_train,y_train,X_val), y_val)
            print(k,'neighbor-' + nor + '-accurcy',acc)
            acc_validation.append(acc)
    
    print("<<<<TEST DATA PREDICTIONS>>>>")
    for k in K:
        for nor in norm:
            acc = evaluation(KNN(k,nor,X_train,y_train,X_test), y_test)
            print(k,'neighbor-' + nor + '-accurcy',acc)
            acc_validation.append(acc)
    ## Complete
    
main()

<<<<VALIDATION DATA PREDICTIONS>>>>
3 neighbor-L1-accurcy 0.92
3 neighbor-L2-accurcy 0.93
3 neighbor-L-inf-accurcy 0.92
5 neighbor-L1-accurcy 0.92
5 neighbor-L2-accurcy 0.94
5 neighbor-L-inf-accurcy 0.91
7 neighbor-L1-accurcy 0.92
7 neighbor-L2-accurcy 0.93
7 neighbor-L-inf-accurcy 0.92
<<<<TEST DATA PREDICTIONS>>>>
3 neighbor-L1-accurcy 0.927536231884058
3 neighbor-L2-accurcy 0.9130434782608695
3 neighbor-L-inf-accurcy 0.9130434782608695
5 neighbor-L1-accurcy 0.927536231884058
5 neighbor-L2-accurcy 0.9420289855072463
5 neighbor-L-inf-accurcy 0.927536231884058
7 neighbor-L1-accurcy 0.9420289855072463
7 neighbor-L2-accurcy 0.9130434782608695
7 neighbor-L-inf-accurcy 0.9130434782608695


Uncomment the code below to run the main function (Remember to recomment the code before submitting).

In [3]:
# Finally, call the main function
# main()

Answer the following questions here:

1. How could having a larger dataset influence the performance of KNN?

2. Tabulate your results from `main()` in the table provided.

3. Finally, mention the best K and the norm combination you have settled upon and report the accuracy on the test set using that combination.

# 1.k和距离的计算
# 2.
## k/dist         3        5       7
## l1            0.927    0.927   0.942
## l2            0.913    0.942   0.913
## l-inf         0.913    0.927   0.913
# 3. k = 5, l2 最好accuracy = 0.942